In [1]:
import os
import pandas as pd

In [2]:
%pwd

'/Users/harkiratchahal/Desktop/Coding/Tutorials/mlops_2/research'

In [3]:
os.chdir('../')

In [11]:
from dataclasses import dataclass
from pathlib import Path

In [18]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    min_samples_split: int
    min_samples_leaf: int
    max_features: str
    max_depth: int
    target_column: str

In [19]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            min_samples_split = params.min_samples_split,
            min_samples_leaf = params.min_samples_leaf,
            max_features = params.max_features,
            max_depth = params.max_depth,
            target_column = schema.name
            
        )

        return model_trainer_config

In [21]:
import os
from mlProject import logger
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import joblib

In [22]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        try:
            train_data = pd.read_csv(self.config.train_data_path)
            test_data = pd.read_csv(self.config.test_data_path)
        except FileNotFoundError as e:
            logger.error(f"File not found: {e}")
            raise
        except pd.errors.EmptyDataError as e:
            logger.error(f"No data: {e}")
            raise
        
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]].values.ravel()
        test_y = test_data[[self.config.target_column]].values.ravel()

        model = RandomForestClassifier(
            n_estimators=self.config.n_estimators,
            min_samples_split=self.config.min_samples_split,
            min_samples_leaf=self.config.min_samples_leaf,
            max_features=self.config.max_features,
            max_depth=self.config.max_depth,
            random_state=42
        )

        try:
            model.fit(train_x, train_y)
        except Exception as e:
            logger.error(f"Failed to train the model: {e}")
            raise

        try:
            joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
            logger.info("Model saved successfully!")
        except Exception as e:
            logger.error(f"Failed to save the model: {e}")
            raise

In [23]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-10-03 17:02:54,317: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-03 17:02:54,321: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-03 17:02:54,323: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-03 17:02:54,325: INFO: common: created directory at: artifacts]
[2023-10-03 17:02:54,326: INFO: common: created directory at: artifacts/model_trainer]
[2023-10-03 17:02:55,249: INFO: 2834237003: Model saved successfully!]
